In [ ]:
import pandas as pd
import random
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, log_loss
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import euclidean
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import chi2_contingency, f_oneway
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold, train_test_split

# Importing Data

In [ ]:
data_train = pd.read_csv('Mushroom_datasets/mushroom_train.csv')
data_test = pd.read_csv('Mushroom_datasets/mushroom_test.csv')
final_accuracy = {}

# Trival System

In [ ]:
N = data_train.shape[0]
N1 = data_train[data_train['class']=='p'].shape[0]
N2 = data_train[data_train['class']=='e'].shape[0]
N_test = data_test['class'].shape[0]
y_pred = []

for _ in range(N_test):
    if random.random() < N1/N:
        y_pred.append('p')
    else:
        y_pred.append('e')


accuracy = accuracy_score(data_test['class'], y_pred)
print("Accuracy of test dataset is", accuracy*100, "%")
final_accuracy['Trival System'] = accuracy*100

f_score = f1_score(data_test['class'], y_pred, pos_label='e')
print("F1 score:", f_score)

# One-Hot Encoding

In [ ]:
features = [col for col in data_train.columns if data_train[col].dtype == 'object']
for i in features:
    print("Unique values of", i, ':', data_train[i].unique())

le = LabelEncoder()
features.remove('class')
temp_train = pd.get_dummies(data_train, columns=features)
temp_test = pd.get_dummies(data_test, columns=features)
temp_train['class'] = le.fit_transform(temp_train['class']) 
temp_test['class'] = le.fit_transform(temp_test['class']) 
print()
print("Extra features added =", temp_train.shape[1]-data_train.shape[1])

### Spliting dataset into train and test set and perfoming standardization

In [ ]:
temp = [col for col in temp_train.select_dtypes(include='float64')]

scaler = StandardScaler()
temp_train[temp] = scaler.fit_transform(temp_train[temp])
X = temp_train.drop('class', axis=1)
y = temp_train['class']

temp_test[temp] = scaler.fit_transform(temp_test[temp])
X_t = temp_test.drop('class', axis=1)
y_t = temp_test['class']

# BaseLine System

In [ ]:
class_means = X.groupby(temp_train['class']).mean()
y_pred = []
for i in range(len(X_t)):
    distances = []
    for j in range(len(class_means)):
        distances.append(euclidean(X_t.iloc[i], class_means.iloc[j]))
    
    y_pred.append(distances.index(min(distances)))
    
accuracy = accuracy_score(y_t, y_pred)
print("Accuracy of test dataset:", accuracy*100, "%")
final_accuracy['Baseline System'] = accuracy*100

f_score = f1_score(y_t, y_pred)
print("F1 score:", f_score)

# Feature Engineering

### Removing features based on correlations w.r.t class column

In [ ]:
temp_train_corr = temp_train[temp_train.columns[1:]].corr(method='pearson')['class'][:]

print("Fetaures whose correlations are more than 10%")
print()
print("Features            Correlation")
print()
temp_train_corr.drop(temp_train_corr[(temp_train_corr.values > -0.1 ) & (temp_train_corr.values < 0.1)].index, inplace=True)
print(temp_train_corr)

new_features = temp_train.columns[~temp_train.columns.isin(temp_train_corr.index)]

### Spliting dataset into train and test set and perfoming standardization

In [ ]:
temp_train = temp_train.drop(new_features, axis=1)
temp_test = temp_test.drop(new_features, axis=1)

X_train = temp_train.drop('class', axis=1)
y_train = temp_train['class']

X_test = temp_test.drop('class', axis=1)
y_test = temp_test['class']

# Models with cross validation (k=10)

In [ ]:
splits = 10
X, X_val, y, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

## 1) Logistic Regression

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    lr = LogisticRegression(max_iter = 10000).fit(train_x, train_y)
    predict_train = lr.predict(train_x)
    train_accuracy.append(accuracy_score(predict_train, train_y))
    
    predict_val = lr.predict(val_x)
    val_accuracy.append(accuracy_score(predict_val, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = lr
        
plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("Logistic Regression")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
y_pred = update.predict(X_test)
final_accuracy['LR'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['LR'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

## 2) Support Vector Machine

In [ ]:
svm = SVC(kernel='linear', C=0.5).fit(X, y)
y_pred = svm.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
svm = SVC(kernel='rbf', C=0.5, gamma = 3).fit(X, y)
y_pred = svm.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
svm = SVC(kernel='rbf', C=3, gamma = 10).fit(X, y)
y_pred = svm.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    svm = SVC(kernel='rbf', C=3, gamma = 10).fit(train_x, train_y)
    predict_train = svm.predict(train_x)
    train_accuracy.append(accuracy_score(predict_train, train_y))
    
    predict_val = svm.predict(val_x)
    val_accuracy.append(accuracy_score(predict_val, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = svm
        
plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("Support Vector Machine")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
y_pred = update.predict(X_test)
final_accuracy['SVM'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['SVM'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

## 3) Naive Bayes

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    nb = GaussianNB().fit(train_x, train_y)
    predict_train = nb.predict(train_x)
    train_accuracy.append(accuracy_score(predict_train, train_y))
    
    predict_val = nb.predict(val_x)
    val_accuracy.append(accuracy_score(predict_val, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = nb

plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("Naive Bayes")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
y_pred = update.predict(X_test)
final_accuracy['Naive Bayes'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['Naive Bayes'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

## 4) Random Forest

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    rfm = RandomForestClassifier(n_estimators=100, random_state=0).fit(train_x, train_y)
    predict_train = rfm.predict(train_x)
    train_accuracy.append(accuracy_score(predict_train, train_y))
    
    predict_val = rfm.predict(val_x)
    val_accuracy.append(accuracy_score(predict_val, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = rfm
        
plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("Random Forest")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
y_pred = update.predict(X_test)
final_accuracy['Random Forest'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['Random Forest'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

## 5) XgBoost

In [ ]:
dtrain = xgb.DMatrix(X, label=y)
params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 3, 'eta': 0.1, 'gamma': 1, 'subsample': 0.8, 'colsample_bytree': 0.8}
num_rounds = 100

xgb_model = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_val, label=y_val)
predict = xgb_model.predict(dtest)
y_pred = [round(y) for y in predict]
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
dtrain = xgb.DMatrix(X, label=y)
params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 8, 'eta': 0.5, 'gamma': 3, 'subsample': 0.8, 'colsample_bytree': 0.8}
num_rounds = 100

xgb_model = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_val, label=y_val)
predict = xgb_model.predict(dtest)
y_pred = [round(y) for y in predict]
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
dtrain = xgb.DMatrix(X, label=y)
params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 15, 'eta': 0.8, 'gamma': 7, 'subsample': 0.8, 'colsample_bytree': 0.8}
num_rounds = 100

xgb_model = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_val, label=y_val)
predict = xgb_model.predict(dtest)
y_pred = [round(y) for y in predict]
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dval = xgb.DMatrix(val_x, label=val_y)
    params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'max_depth': 8, 'eta': 0.5, 'gamma': 3, 'subsample': 0.8, 'colsample_bytree': 0.8}
    num_rounds = 100
    
    xgb_model = xgb.train(params, dtrain, num_rounds)
    y_pred_train = xgb_model.predict(dtrain)
    y_pred_train_binary = [round(y) for y in y_pred_train]
    train_accuracy.append(accuracy_score(y_pred_train_binary, train_y))
    
    y_pred_test = xgb_model.predict(dval)
    y_pred_test_binary = [round(y) for y in y_pred_test]
    val_accuracy.append(accuracy_score(y_pred_test_binary, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = xgb_model
        
plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("XgBoost")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
dtest = xgb.DMatrix(X_test, label=y_test)
predict = update.predict(dtest)
y_pred = [round(y) for y in predict]
final_accuracy['XgBoost'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['XgBoost'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

## 6) Multi Layer Perceptron

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, learning_rate_init=0.001, max_iter=10000, early_stopping=False) 
mlp.fit(X, y)

y_pred = mlp.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', solver='sgd', alpha=0.001, learning_rate_init=0.01, max_iter=10000, early_stopping=False) 
mlp.fit(X, y)

y_pred = mlp.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', solver='adam', alpha=0.01, learning_rate_init=0.5, max_iter=10000, early_stopping=False) 
mlp.fit(X, y)

y_pred = mlp.predict(X_val)
print(accuracy_score(y_pred, y_val))

f_score = f1_score(y_val, y_pred)
print("F1 score:", f_score)

In [ ]:
kfold = KFold(n_splits=splits, shuffle=True)
train_accuracy = []
val_accuracy = []
acc = 0

for i,j in kfold.split(X_train):
    train_x, val_x = X_train.iloc[i], X_train.iloc[j]
    train_y, val_y = y_train.iloc[i], y_train.iloc[j]
    
    mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, learning_rate_init=0.001, max_iter=10000, early_stopping=False) 
    mlp.fit(train_x, train_y)
    predict_train = mlp.predict(train_x)
    train_accuracy.append(accuracy_score(predict_train, train_y))
    
    predict_val = mlp.predict(val_x)
    val_accuracy.append(accuracy_score(predict_val, val_y))
    
    if val_accuracy[-1] > acc:
        acc = val_accuracy[-1]
        update = mlp
        
plt.plot(range(1, splits+1), np.array(train_accuracy)*100, label="Train dataset")
plt.plot(range(1, splits+1), np.array(val_accuracy)*100, label="Val dataset")
plt.title("Multi Layer Perceptron")
plt.xlabel('KFold Splits')
plt.ylabel('Accuracy(%)')
plt.legend()

In [ ]:
y_pred = update.predict(X_test)
final_accuracy['MLP'] = accuracy_score(y_pred, y_test)*100
print("Accuracy of test dataset:", final_accuracy['MLP'], "%")

f_score = f1_score(y_test, y_pred)
print("F1 score:", f_score)

# Final accuracies for all models

In [ ]:
accuracy = list(final_accuracy.keys())
models = list(final_accuracy.values())
plt.xticks(rotation = 45, ha='right')

bars = plt.bar(accuracy, models)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x()+bar.get_width()/2, height, '{:.3f}'.format(height), ha='center', va='bottom')
    
plt.title("Model Accuracies for Reduced Feature Engineering")
plt.ylabel("Accuracy(%)")
plt.show()

In [ ]:
best_model = max(final_accuracy, key=final_accuracy.get)
best_accuracy = final_accuracy[best_model]
print(f"Best model is {best_model} with accuracy {best_accuracy} %")